# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235338472629                   -0.50    8.0
  2   -7.250306319304       -1.82       -1.41    1.0
  3   -7.251298919778       -3.00       -2.11    4.0
  4   -7.251212277069   +   -4.06       -2.11    3.0
  5   -7.251327507669       -3.94       -2.63    2.0
  6   -7.251338236777       -4.97       -3.22    2.0
  7   -7.251338783091       -6.26       -4.10    2.0
  8   -7.251338793076       -8.00       -4.16    5.0
  9   -7.251338798075       -8.30       -4.61    2.0
 10   -7.251338798633       -9.25       -5.12    2.0
 11   -7.251338798701      -10.16       -5.68    3.0
 12   -7.251338798704      -11.50       -6.55    3.0
 13   -7.251338798705      -13.09       -6.99    3.0
 14   -7.251338798705      -13.94       -7.46    3.0
 15   -7.251338798705      -15.05       -8.11    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04876027196376789
[ Info: Arnoldi iteration step 2: normres = 0.2765761933085702
[ Info: Arnoldi iteration step 3: normres = 0.23754028606193606
[ Info: Arnoldi iteration step 4: normres = 0.616429419947394
[ Info: Arnoldi iteration step 5: normres = 0.2621754636925592
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.38e-01, 5.54e-02, 1.68e-01, 1.19e-01, 6.47e-02)
[ Info: Arnoldi iteration step 6: normres = 1.0504121757668388
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.22e-01, 2.01e-01, 9.19e-01, 1.53e-01, 5.48e-02)
[ Info: Arnoldi iteration step 7: normres = 0.13781399768724534
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.01e-01, 1.81e-02, 6.93e-03, 7.35e-02, 4.67e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12880402398233526
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.77e-0